In [1]:
import pandas as pd
import numpy as np
import pydub
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import pickle
from multiprocessing import Pool
import os
from datetime import datetime
from collections import namedtuple
from itertools import combinations
import random

Range = namedtuple('Range',['start','end'])
os.chdir('/project2/mbspencer/nih/transcripts/')

#### For each file: Use intersection of transcriber utterance timings as threshold to create union of transcriber utterance timings, then use union to clip audio - add silence between, save file, receive test audio corpus with manual VAD feature extraction

In [2]:
df_transcripts = pd.read_csv('transcripts.csv')
df_transcripts.head()

,feed,zone,year,month,day,time,sid,start,end,transcription,notes,file,transcriber,start_c,start_dt,end_c,end_dt,length,length_s
0,27730,Zone1,2018,8,12,932,RADIO,00.02.21.252,00.02.31.279,RADIOSHOP TESTING ONE TWO THREE FOUR FIVE FIVE...,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.21.252,1900-01-01 00:02:21.252000,00.02.31.279,1900-01-01 00:02:31.279000,0 days 00:00:10.027000000,10.027
1,27730,Zone1,2018,8,12,932,RADIO,00.02.38.109,00.02.39.417,ONE TWO ONE TWO,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.38.109,1900-01-01 00:02:38.109000,00.02.39.417,1900-01-01 00:02:39.417000,0 days 00:00:01.308000000,1.308
2,27730,Zone1,2018,8,12,932,UNIT,00.02.48.327,00.02.49.235,UNIT COMING IN,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.48.327,1900-01-01 00:02:48.327000,00.02.49.235,1900-01-01 00:02:49.235000,0 days 00:00:00.908000000,0.908
3,27730,Zone1,2018,8,12,932,UNIT,00.02.55.330,00.02.57.437,ZONE ONE IS ON CITY [WIDE] FIVE,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.55.330,1900-01-01 00:02:55.330000,00.02.57.437,1900-01-01 00:02:57.437000,0 days 00:00:02.107000000,2.107
4,27730,Zone1,2018,8,12,932,RADIO,00.03.04.003,00.03.09.017,OKAY THANKS UH THIS IS THE RADIO SHOP TESTING ...,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.03.04.003,1900-01-01 00:03:04.003000,00.03.09.017,1900-01-01 00:03:09.017000,0 days 00:00:05.014000000,5.014


In [3]:
df_transcripts[['filename','transcriber']] = df_transcripts['file'].str.split('(\d*-\d*-\d*)',expand=True)[[1,2]]
def remove_filetype(x):
    y = x.replace('.xlsx','')
    y = y.replace('.txt','')
    return y
df_transcripts['transcriber'] = df_transcripts['transcriber'].apply(lambda x: remove_filetype(x))
df_transcripts['transcriber'] = df_transcripts['transcriber'].str.lower().str.strip()
df_transcripts = df_transcripts[df_transcripts['transcriber']!='xxx']
#df_transcripts[['filename','transcriber']]
df_transcripts['transcriber'].value_counts()

sj     5303
his    3703
mgw    3380
tjs    3262
wpd    2893
mh     2489
kjc    1565
wam    1416
aek    1387
ah     1376
ahs     763
rc      674
mcm     582
oaa     515
rah     501
jar     478
kk      353
kw      223
Name: transcriber, dtype: int64

In [4]:
df_transcripts['start_dt'] = pd.to_datetime(df_transcripts['start_dt'])
df_transcripts['end_dt'] = pd.to_datetime(df_transcripts['end_dt'])

In [5]:
def remove_irrelevant(x):
    x = str(x)
    x = x.replace("\[Uncertain\]","")
    x = x.replace("INAUDIBLE","")    
    x = x.replace("<X>","")
    x = ' '.join(x.split())
    x = ''.join([i for i in x if i.isalnum() or i==' '])
    x = x.strip()
    x = x.upper()
    return x

In [6]:
df_transcripts['start_ts'] = (df_transcripts['start_dt']-np.datetime64('1900-01-01T00:00:00.000000000')).dt.total_seconds()
df_transcripts['end_ts'] = (df_transcripts['end_dt']-np.datetime64('1900-01-01T00:00:00.000000000')).dt.total_seconds()

In [7]:
df_transcripts['Range'] = df_transcripts[['start_ts','end_ts']].apply(tuple,axis=1)
df_transcripts.head()

,feed,zone,year,month,day,time,sid,start,end,transcription,...,start_c,start_dt,end_c,end_dt,length,length_s,filename,start_ts,end_ts,Range
0,27730,Zone1,2018,8,12,932,RADIO,00.02.21.252,00.02.31.279,RADIOSHOP TESTING ONE TWO THREE FOUR FIVE FIVE...,...,00.02.21.252,1900-01-01 00:02:21.252,00.02.31.279,1900-01-01 00:02:31.279,0 days 00:00:10.027000000,10.027,201808120932-28710-27730,141.252,151.279,"(141.252, 151.279)"
1,27730,Zone1,2018,8,12,932,RADIO,00.02.38.109,00.02.39.417,ONE TWO ONE TWO,...,00.02.38.109,1900-01-01 00:02:38.109,00.02.39.417,1900-01-01 00:02:39.417,0 days 00:00:01.308000000,1.308,201808120932-28710-27730,158.109,159.417,"(158.109, 159.417)"
2,27730,Zone1,2018,8,12,932,UNIT,00.02.48.327,00.02.49.235,UNIT COMING IN,...,00.02.48.327,1900-01-01 00:02:48.327,00.02.49.235,1900-01-01 00:02:49.235,0 days 00:00:00.908000000,0.908,201808120932-28710-27730,168.327,169.235,"(168.327, 169.235)"
3,27730,Zone1,2018,8,12,932,UNIT,00.02.55.330,00.02.57.437,ZONE ONE IS ON CITY [WIDE] FIVE,...,00.02.55.330,1900-01-01 00:02:55.330,00.02.57.437,1900-01-01 00:02:57.437,0 days 00:00:02.107000000,2.107,201808120932-28710-27730,175.330,177.437,"(175.33, 177.437)"
4,27730,Zone1,2018,8,12,932,RADIO,00.03.04.003,00.03.09.017,OKAY THANKS UH THIS IS THE RADIO SHOP TESTING ...,...,00.03.04.003,1900-01-01 00:03:04.003,00.03.09.017,1900-01-01 00:03:09.017,0 days 00:00:05.014000000,5.014,201808120932-28710-27730,184.003,189.017,"(184.00300000000001, 189.01700000000002)"


In [8]:
def get_prop_overlap(se1, se2):
    s1, e1 = se1
    s2, e2 = se2
    overlap = max(0,min(e1,e2)-max(s1,s2))
    prop_overlap = overlap/max(e1-s1,e2-s2)
    return prop_overlap

In [9]:
def get_se_voice(filename, df_transcripts):
    df = df_transcripts[df_transcripts['filename']==f].copy()
    # Extract information so we can reconstruct dir_path
    zone = df['zone'].unique()[0]
    year = df['year'].astype(str).unique()[0]
    month = df['month'].astype(str).apply(lambda x: x.zfill(2)).unique()[0]
    day = df['day'].astype(str).apply(lambda x: x.zfill(2)).unique()[0]
    date = year+'_'+month+'_'+day
    # Create a list to hold the ranges we derive
    se_voice_list = []
    # Create a list of "exhausted" tuples (i.e. do not try to compare to others)
    exhausted = []
    # Get transcribers
    transcribers = df['transcriber'].unique().tolist()
    # If only 1 transcription...
    # If 2+ transcriptions...
    if len(transcribers)>=2:
        # Pick 2 transcribers at random to avoid "issues"
        df_foruse = df[df['transcriber'].isin(random.sample(transcribers,2))].copy()
        # Get the ranges
        ranges = df_foruse['Range'].values
        # Check all permutations
        checked_se_tuples = []
        # Get target range
        for se1 in ranges:
            # Collect candidates
            candidate_se_list = []
            # Get start/end for target range
            s1, e1 = se1
            # Check overlap with remaining ranges
            for se2 in ranges:
                se_tuple = set((se1, se2))
                # Make sure it's not the target range and we have not checked for overlap of these ranges
                if se1!=se2: #) and len([i for i in checked_se_tuples if se_tuple in i])==0:
                    #checked_se_tuples.append(se_tuple)
                    prop_overlap = get_prop_overlap(se1, se2)
                    # If overlap more than 75%
                    if prop_overlap > 0.75 and [se2, se1] not in checked_se_tuples:
                        checked_se_tuples.append([se1, se2])
                        exhausted.append(se2)
                        exhausted.append(se1)
                        # Get start/end for other range
                        s2, e2 = se2
                        # Get the minimum start time of both periods 
                        start_voice = min(s1,s2)
                        # Get the maximum end time of both periods
                        end_voice = max(e1,e2)
                        # Get transcriptions associated with each range
                        t1 = df_foruse.loc[df['Range']==se1,'transcription'].apply(remove_irrelevant).values[0]
                        t2 = df_foruse.loc[df['Range']==se2,'transcription'].apply(remove_irrelevant).values[0]
                        # Add to candidate list
                        candidate_se_list.append([zone, date, f+'.mp3', start_voice, end_voice, t1==t2, t1, t2])
            if len(candidate_se_list)>0:
                se_voice_list.append(candidate_se_list[0])
    return se_voice_list

In [22]:
'''
        # If only one overlapping range is found then add it to the list of voice activity for the file
        if len(candidate_se_list)==1:
            se_voice_list.append(candidate_se_list[0])
        # If more than one overlapping range...
        if len(candidate_se_list)>1:
            # Add the first overlapping range (NOTE: THIS IS NOT IDEAL)
            se_voice_list.append(candidate_se_list[0])
'''


'\n        # If only one overlapping range is found then add it to the list of voice activity for the file\n        if len(candidate_se_list)==1:\n            se_voice_list.append(candidate_se_list[0])\n        # If more than one overlapping range...\n        if len(candidate_se_list)>1:\n            # Add the first overlapping range (NOTE: THIS IS NOT IDEAL)\n            se_voice_list.append(candidate_se_list[0])\n'

In [10]:
%%time
voice_list = []
for f in df_transcripts['filename'].unique():
    voice_list += get_se_voice(f,df_transcripts)

CPU times: user 13 s, sys: 34.3 ms, total: 13 s
Wall time: 13 s


In [11]:
df_va = pd.DataFrame(voice_list,columns=['zone','date','filename','start_voice','end_voice','transcripts_agree','transcription1','transcription2'])
df_va = df_va.drop_duplicates()
df_va.sort_values(['zone','date','start_voice'],inplace=True)
df_va.to_csv('df_va.csv')
df_va.head(20)

,zone,date,filename,start_voice,end_voice,transcripts_agree,transcription1,transcription2
2601,Zone1,2018_08_04,201808042331-339616-27730.mp3,70.952,73.086,True,SEVENTEEN TWENTY FOUR ROBERT,SEVENTEEN TWENTY FOUR ROBERT
2602,Zone1,2018_08_04,201808042331-339616-27730.mp3,73.712,75.073,True,TWENTY FOUR ROBERT,TWENTY FOUR ROBERT
2603,Zone1,2018_08_04,201808042331-339616-27730.mp3,75.597,81.454,False,RESIDENTIAL ALARM FOURTY THIRTY ONE PATTERSON ...,RESIDENTIAL ALARM FORTY THIRTY ONE PATTERSON P...
2604,Zone1,2018_08_04,201808042331-339616-27730.mp3,81.790,82.598,True,TEN FOUR,TEN FOUR
2605,Zone1,2018_08_04,201808042331-339616-27730.mp3,90.744,92.483,False,PAGING PAGING ROBERT,SEVENTEEN THIRTEEN ROBERT
2606,Zone1,2018_08_04,201808042331-339616-27730.mp3,93.638,95.388,True,SEVENTEEN THIRTEEN ROBERT,SEVENTEEN THIRTEEN ROBERT
2607,Zone1,2018_08_04,201808042331-339616-27730.mp3,96.398,100.189,False,I NEED A RESULT FOR A BOOKING ON ARAGON ALBANY,I CAN COOL DOWN FOR A PROTOCOL ON EISENHOWER A...
2608,Zone1,2018_08_04,201808042331-339616-27730.mp3,118.949,122.450,True,SIXTEEN THIRTY FOUR ROBERT,SIXTEEN THIRTY FOUR ROBERT
2609,Zone1,2018_08_04,201808042331-339616-27730.mp3,152.204,157.320,False,SEVENTEEN TWENTY THREE ROBERT YOUR CASE SON IS...,SEVENTEEN TWENTY THREE ROBERT YOUR CASE HUN IS...
2610,Zone1,2018_08_04,201808042331-339616-27730.mp3,252.242,255.165,True,SIXTEEN FOURTEEN ROBERT,SIXTEEN FOURTEEN ROBERT


### This code for file-by-file removal of silence using manually provided VAD data

In [12]:
# Set some defaults 
silence_buffer = 2500 # ms
num_processors = 28
base_path = '/project2/graziul/LoP/corpus/transcriptions/WorkingData/'

In [13]:
def get_all_voice_activity(df_va):
    p = Pool(processes = num_processors)
    output = p.starmap(extract_voice_activity,[(file,df_va) for file in df_va['filename'].unique()])
    p.close()

In [15]:
def extract_voice_activity(file, df_va, padding=200):
    # Get data associated with file
    df_temp = df_va[df_va['filename']==file].copy()
    # Get data needed to construct path to file
    zone = df_temp['zone'].unique()[0]
    date = df_temp['date'].unique()[0]
    # Load file
    try:
        mp3_file_in = base_path+zone+'/'+date+'/'+file
        pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
    except:
        date_split = date.split('_')
        day = int(date_split[2])+1
        date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
        mp3_file_in = base_path+zone+'/'+date+'/'+file
        pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
    # Get frame_rate
    fr = pydub_audiosegment.frame_rate
    # Create silence
    silence = AudioSegment.silent(duration = silence_buffer, frame_rate = fr)
    # Get (start, end) tuples to extract voice activity
    se = df_temp[['start_voice','end_voice']].sort_values('start_voice').values
    # Build up list of audio segments with voice activity  
    va_slices = []
    for s,e in se: 
        va_slices.append(pydub_audiosegment[max(0,s*1000-padding):min(e*1000+padding,len(pydub_audiosegment))])
    combined = silence
    for va_slice in va_slices:
        combined += va_slice + silence
    wav_file_out = base_path+zone+'/'+date+'/'+file.split('.')[0]+'va.wav'
    combined = combined.set_frame_rate(fr)
    combined.export(wav_file_out,format='wav')

In [16]:
%%time
get_all_voice_activity(df_va)

CPU times: user 445 ms, sys: 191 ms, total: 636 ms
Wall time: 15.2 s


### This code for creating test files using manually provided VAD data

In [49]:
df_va = pd.read_csv('df_va.csv')
df_va.drop(columns=df_va.columns.values[0],inplace=True)
df_va.head(10)

,zone,date,filename,start,end,transcription1,transcription2,transcripts_agree
0,Zone1,2018_08_12,201808120932-28710-27730.mp3,168.308,169.235,UNIT COMING IN,UNIT COMING IN,True
1,Zone1,2018_08_12,201808120932-28710-27730.mp3,175.330,177.437,ZONE ONE IS ON CITY WIDE FIVE,ZONE ONE IS ON CITY WIDE FIVE,True
2,Zone1,2018_08_12,201808120932-28710-27730.mp3,184.003,189.017,OKAY THANKS UH THIS IS THE RADIO SHOP TESTING ...,OKAY THANKS THIS IS THE RADIO SHOP TESTING HOW...,False
3,Zone1,2018_08_12,201808120932-28710-27730.mp3,231.627,235.020,ONE TWO TRES CUATRO,UNO DOS ONE TWO THREE CUATRO,False
4,Zone1,2018_08_12,201808120932-28710-27730.mp3,850.490,851.798,NaN,TEEN OH SEVEN DAVID,False
5,Zone1,2018_08_12,201808120932-28710-27730.mp3,856.303,861.135,SIXTEEN OH SEVEN DAVID THE ZONE ONE FIVE IS ON...,SIXTEEN OH SEVEN DAVID THE ZONE IS ON CITYWIDE...,False
6,Zone1,2018_08_12,201808120932-28710-27730.mp3,1022.968,1024.450,RADIOSHOP TEST ONE,RADIO SHOP TEST ZONE ONE,False
7,Zone1,2018_08_12,201808120932-28710-27730.mp3,1028.908,1030.106,OH NINE FOURTY NINE,OH NINE FORTY NINE,False
8,Zone1,2018_08_12,201808120932-28710-27730.mp3,1041.768,1044.617,UNO DOS TRES CUATRO CINCO LOUD AND CLEAR,UNO DOS TRES CUATRO CINCO LOUD AND CLEAR,True
9,Zone1,2018_08_12,201808120932-28710-27730.mp3,1083.712,1088.525,LOUD AND CLEAR ONE TWO THREE FOUR FIVE,LOUD AND CLEAR ONE TWO THREE FOUR FIVE,True


In [ ]:
# Now use these timings to compile sample data for use by GMR and Deepgram
#  1. 3 x 10 min files for GMR
#  2. "a couple hours" for Deepgram
#  3. Preserve temporal order

In [28]:
# File #1: Both transcribers agree ("clean" = easy)
df_va1 = df_va[df_va['transcripts_agree']].copy()
df_va1 = df_va1.drop_duplicates()
df_va1 = df_va1[~df_va1['transcription1'].isna()]
len(df_va1)

1629

In [29]:
df_va1.sort_values(['date','start_voice'],inplace=True)
# Exhausted case indices
exhausted_cases = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = df_va1['filename'].unique().tolist()
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va1[df_va1['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #1 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    exhausted_cases.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = base_path+'test1.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

File #1 now 0 in length 201808042331-339616-27730.mp3
File #1 now 0.1 in length 201808042331-339616-27730.mp3
File #1 now 0.2 in length 201808042331-339616-27730.mp3
File #1 now 0.3 in length 201808042331-339616-27730.mp3
File #1 now 0.3 in length 201808042331-339616-27730.mp3
File #1 now 0.4 in length 201808042331-339616-27730.mp3
File #1 now 0.5 in length 201808042331-339616-27730.mp3
File #1 now 0.6 in length 201808042331-339616-27730.mp3
File #1 now 0.7 in length 201808042331-339616-27730.mp3
File #1 now 0.8 in length 201808042331-339616-27730.mp3
File #1 now 0.8 in length 201808042331-339616-27730.mp3
File #1 now 0.9 in length 201808042331-339616-27730.mp3
File #1 now 1.0 in length 201808042331-339616-27730.mp3
File #1 now 1.1 in length 201808042331-339616-27730.mp3
File #1 now 1.2 in length 201808042331-339616-27730.mp3
File #1 now 1.3 in length 201808042331-339616-27730.mp3
File #1 now 1.5 in length 201808042331-339616-27730.mp3
File #1 now 1.6 in length 201808042331-339616-2773

<_io.BufferedRandom name='/project2/graziul/LoP/corpus/transcriptions/WorkingData/test1.wav'>

In [56]:
exhausted_from_test1 = pd.Index([i for s in exhausted_cases for i in s])
len(exhausted_cases), len(df_va), len(df_va[~df_va.index.isin(exhausted_from_test1)])

(126, 4764, 4638)

In [106]:
df_va[df_va.index.isin(exhausted_from_test1)].to_csv('test1.csv')

In [105]:
# File #2: Mix of agree/disagree ("not clean" = medium)
df_va2 = df_va[(~df_va.index.isin(exhausted_from_test1)) & (df_va.index>max(exhausted_from_test1))].copy()
print("Proportion of samples where transcripts disagree:", str(len(df_va2[~df_va2['transcripts_agree']])/len(df_va2)))
print("Length of df_va2: ",str(len(df_va2)))

Proportion of samples where transcripts disagree: 0.6970976253298153
Length of df_va2:  1895


In [72]:
df_va2.sort_values(['date','start_voice'],inplace=True)
df_va2.head(20)

,zone,date,filename,start_voice,end_voice,transcripts_agree,transcription1,transcription2
3470,Zone1,2018_08_05,201808050200-579833-27730.mp3,11.313,14.092,True,SEVENTEEN THIRTY ONE ROBERT,SEVENTEEN THIRTY ONE ROBERT
3471,Zone1,2018_08_05,201808050200-579833-27730.mp3,14.818,15.653,False,TWO ONE ROBERT,ONE ROBERT
3472,Zone1,2018_08_05,201808050200-579833-27730.mp3,17.324,33.940,False,COMING IN AS A PERSON WITH A KNIFE FORTY FORTY...,COMING IN A PERSON WITH A KNIFE FORTY FORTY SE...
2960,Zone1,2018_08_05,201808050828-792675-27730.mp3,17.495,18.882,False,SEVENTEEN TWENTY,TEEN TWENTY
2961,Zone1,2018_08_05,201808050828-792675-27730.mp3,20.272,21.545,True,SEVEN TWENTY,SEVEN TWENTY
2962,Zone1,2018_08_05,201808050828-792675-27730.mp3,21.902,29.960,False,WE HAVE CLEARANCE FOR US BUT SEND SEVENTEEN TW...,WE GOT A STATE REPORT OCCUPIED TIMES SEVENTEEN...
2874,Zone1,2018_08_05,201808050100-298054-27730.mp3,28.467,30.726,True,SIXTEEN TWENTY FOUR ROBERT,SIXTEEN TWENTY FOUR ROBERT
2963,Zone1,2018_08_05,201808050828-792675-27730.mp3,31.347,32.475,True,SEVEN TWELVE,SEVEN TWELVE
2964,Zone1,2018_08_05,201808050828-792675-27730.mp3,34.473,35.086,False,YUP,TWELVE
3473,Zone1,2018_08_05,201808050200-579833-27730.mp3,35.919,38.048,False,TEN FOUR WHAT WERE THOSE NUMBERS ON MELROSE AGAIN,TEN FOUR WHAT ARE THE NUMBERS ON MELROSE AGAIN


In [74]:
df_va2.sort_values(['date','start_voice'],inplace=True)
# Exhausted case indices
exhausted_cases = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = df_va2['filename'].unique().tolist()
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va2[df_va2['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #2 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    exhausted_cases.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = base_path+'test2.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

File #2 now 0 in length 201808050001-100238-27730.mp3
File #2 now 0.2 in length 201808050001-100238-27730.mp3
File #2 now 0.4 in length 201808050100-298054-27730.mp3
File #2 now 0.5 in length 201808050100-298054-27730.mp3
File #2 now 0.6 in length 201808050100-298054-27730.mp3
File #2 now 1.0 in length 201808050100-298054-27730.mp3
File #2 now 1.1 in length 201808050100-298054-27730.mp3
File #2 now 1.1 in length 201808050100-298054-27730.mp3
File #2 now 1.2 in length 201808050100-298054-27730.mp3
File #2 now 1.4 in length 201808050100-298054-27730.mp3
File #2 now 1.5 in length 201808050100-298054-27730.mp3
File #2 now 1.6 in length 201808050100-298054-27730.mp3
File #2 now 1.7 in length 201808050100-298054-27730.mp3
File #2 now 1.7 in length 201808050100-298054-27730.mp3
File #2 now 1.8 in length 201808050100-298054-27730.mp3
File #2 now 2.0 in length 201808050100-298054-27730.mp3
File #2 now 2.0 in length 201808050100-298054-27730.mp3
File #2 now 2.3 in length 201808050100-298054-2773

<_io.BufferedRandom name='/project2/graziul/LoP/corpus/transcriptions/WorkingData/test2.wav'>

In [82]:
exhausted_from_test2 = pd.Index([i for s in exhausted_cases for i in s])
len(exhausted_from_test1), len(exhausted_from_test2), len(df_va), len(df_va2), len(df_va[~df_va.index.isin(exhausted_from_test1.union(exhausted_from_test2))])

(126, 93, 4764, 1895, 4545)

In [103]:
df_va[df_va.index.isin(exhausted_from_test2)].to_csv('test2.csv')

In [92]:
# File #3: Transcribers disagree ("not clean" = hard)'
df_va3 = df_va[(~df_va['transcripts_agree']) & (df_va.index>max(exhausted_from_test2)-500)].copy()
print("Proportion of samples where transcripts disagree:", str(len(df_va3[~df_va3['transcripts_agree']])/len(df_va3)))
print("Length of df_va3: ",str(len(df_va3)))

Proportion of samples where transcripts disagree: 1.0
Length of df_va3:  1265


In [93]:
df_va3.sort_values(['date','start_voice'],inplace=True)
df_va3.head(20)

,zone,date,filename,start_voice,end_voice,transcripts_agree,transcription1,transcription2
3471,Zone1,2018_08_05,201808050200-579833-27730.mp3,14.818,15.653,False,TWO ONE ROBERT,ONE ROBERT
3472,Zone1,2018_08_05,201808050200-579833-27730.mp3,17.324,33.940,False,COMING IN AS A PERSON WITH A KNIFE FORTY FORTY...,COMING IN A PERSON WITH A KNIFE FORTY FORTY SE...
3473,Zone1,2018_08_05,201808050200-579833-27730.mp3,35.919,38.048,False,TEN FOUR WHAT WERE THOSE NUMBERS ON MELROSE AGAIN,TEN FOUR WHAT ARE THE NUMBERS ON MELROSE AGAIN
3477,Zone1,2018_08_05,201808050200-579833-27730.mp3,47.255,51.791,False,TAKE A RIDE ON THAT FORTY FORTY SEVEN MELROSE ...,TAKE A RIGHT ON THE FORTY FORTY SEVEN MELROSE ...
3540,Zone1,2018_08_05,201808050658-982393-27730.mp3,83.251,88.501,False,SIX SEVEN ONE UH WERE AT TWENTY FIVE MILEAGE O...,SIX SEVEN ONE ROBERT UH DIAL MILEAGE ONE SEVEN...
2976,Zone1,2018_08_05,201808050828-792675-27730.mp3,133.443,142.374,False,THIS WAY AND IVE GOTTA GET UP TO NINETY FIVE C...,AND UM I GOTTA PICK UP SOME NINETY FIVES BECAU...
3398,Zone1,2018_08_05,201808050758-615610-27730.mp3,165.883,167.381,False,THANK YOU AND GOOD MORNING,THANK YOU MAAM GOOD MORNING
2980,Zone1,2018_08_05,201808050828-792675-27730.mp3,167.742,170.313,False,WE ARE CHANGING LOCATION TO FOREST HOSPITAL,WE GOT A CHANGE IN LOCATION TO HOSPITAL
3419,Zone1,2018_08_05,201808050230-338958-27730.mp3,173.806,177.115,False,HERES WHAT I CAN GET TO YOU,THIRTY ONE HUNDRED THREE ONE NINETY TWO
3421,Zone1,2018_08_05,201808050230-338958-27730.mp3,192.079,193.318,False,NOISE,TWENTY ROBERT


In [94]:
df_va3.sort_values(['date','start_voice'],inplace=True)
# Exhausted case indices
exhausted_cases = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = df_va3['filename'].unique().tolist()
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va3[df_va3['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #3 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    exhausted_cases.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = base_path+'test3.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

File #3 now 0 in length 201808050200-579833-27730.mp3
File #3 now 0.1 in length 201808050200-579833-27730.mp3
File #3 now 0.4 in length 201808050200-579833-27730.mp3
File #3 now 0.5 in length 201808050200-579833-27730.mp3
File #3 now 0.6 in length 201808050200-579833-27730.mp3
File #3 now 0.7 in length 201808050200-579833-27730.mp3
File #3 now 0.9 in length 201808050200-579833-27730.mp3
File #3 now 1.1 in length 201808050200-579833-27730.mp3
File #3 now 1.2 in length 201808050200-579833-27730.mp3
File #3 now 1.4 in length 201808050200-579833-27730.mp3
File #3 now 1.5 in length 201808050200-579833-27730.mp3
File #3 now 1.6 in length 201808050200-579833-27730.mp3
File #3 now 1.6 in length 201808050200-579833-27730.mp3
File #3 now 1.7 in length 201808050200-579833-27730.mp3
File #3 now 1.8 in length 201808050200-579833-27730.mp3
File #3 now 1.9 in length 201808050200-579833-27730.mp3
File #3 now 2.1 in length 201808050200-579833-27730.mp3
File #3 now 2.1 in length 201808050200-579833-2773

<_io.BufferedRandom name='/project2/graziul/LoP/corpus/transcriptions/WorkingData/test3.wav'>

In [100]:
exhausted_from_test3 = pd.Index([i for s in exhausted_cases for i in s])

In [102]:
df_va[df_va.index.isin(exhausted_from_test3)].to_csv('test3.csv')